In [5]:
import pandas as pd
import re
from sklearn.metrics import f1_score, cohen_kappa_score
from tqdm import tqdm

In [6]:
def map_fn(row):
    row = str(row)
    if(re.search('yes', row) and re.search('no', row)):
        return None
    elif(re.search('yes', row)):
        return 1
    elif(re.search('no', row)):
        return 0
    elif(re.search('hateful', row)):
        return 1
    elif(re.search('hate.', row)):
        return 1

    return None

In [7]:
files = ['PA_base'] + ['PA_'+str(i) for i in range(1,10)] + ['PF_base'] + ['PF_'+str(i) for i in range(1,10)] + ['PG_base'] + ['PG_'+str(i) for i in range(1,10)] + ['PH_base'] + ['PH_'+str(i) for i in range(1,10)]

for f in tqdm(files):
    df = pd.read_excel('../ChatGPT/'+f+'.xlsx')
    col = df.columns[2:]
    
    no_ans = []
    F1 = []
    IAA = []
    HS = []
    
    for i in col:
        df[i] = df[i].apply(lambda x: str(x).lower())
        df[i] = df[i].apply(map_fn)

        pen = (df.shape[0]-df[i].isna().sum())/df.shape[0]
        no_ans.append(df[i].isna().sum())

        tmp = df[['label', i]].dropna()

        F1.append(pen*f1_score(tmp['label'], tmp[i], average='weighted'))
        IAA.append(pen*cohen_kappa_score(tmp['label'], tmp[i]))
        HS.append(tmp[tmp[i]==1].shape[0]/tmp.shape[0])

    
    res = pd.read_csv('Performance.csv')
    res[f+' Hal'] = no_ans
    res[f+' F1'] = F1
    res[f+' IAA'] = IAA
    res[f+' HS'] = HS
    res.to_csv('Performance.csv', index=False)

  0%|          | 0/40 [00:00<?, ?it/s]

100%|██████████| 40/40 [00:04<00:00,  9.54it/s]


In [8]:
res

,Model,PA_base Hal,PA_base F1,PA_base IAA,PA_base HS,PA_1 Hal,PA_1 F1,PA_1 IAA,PA_1 HS,PA_2 Hal,...,PH_7 IAA,PH_7 HS,PH_8 Hal,PH_8 F1,PH_8 IAA,PH_8 HS,PH_9 Hal,PH_9 F1,PH_9 IAA,PH_9 HS
0,gpt,0,0.787792,0.58,0.398,0,0.829967,0.66,0.486,0,...,0.031768,0.014,0,0.378382,0.009089,0.004,0,0.438925,0.068941,0.038
